In [ ]:
import pandas as pd
import polars as pl

## LUAD

In [ ]:
#inspection for all data types
url = r'Z:\HiWi\Popp\TCGA_NSCLC_2022\LUAD\DNA_methylation\TCGA-LUAD.methylation450.tsv' # r"Z:\HiWi\Popp\TCGA_Breast_2022\TCGA-BRCA.methylation450.tsv" # r'Z:\HiWi\Popp\TCGA_NSCLC_2022\LUAD\DNA_methylation\TCGA-LUAD.methylation450.tsv'
df_LUAD = pl.read_csv(url, sep='\t')
#df_LUAD = pd.read_csv(url, sep='\t')

In [ ]:
#df_LUAD.dropna(how='all', axis=0, inplace=True) #drop patient
df_LUAD = df_LUAD.drop_nulls()
df_LUAD = df_LUAD.to_pandas()
df_LUAD.set_index('Composite Element REF', inplace=True)
df = df_LUAD
df_LUAD

## LUSC

In [ ]:
#inspection for all data types
url = r'Z:\HiWi\Popp\TCGA_NSCLC_2022\LUSC\DNA_methylation\TCGA-LUSC.methylation450.tsv'
df_LUSC = pd.read_csv(url, sep='\t', index_col=0)
df_LUSC.dropna(how='all', axis=0, inplace=True) #drop patient
df_LUSC

In [ ]:
#combine LUAD and LUSC
if list(df_LUAD.index) == list(df_LUSC.index): #if same order
    df = pd.concat([df_LUAD, df_LUSC], axis = 1) 
df

In [ ]:
df.dropna(how='all', axis=0, inplace=True)
df.dropna(how='all', axis=1, inplace=True)
print(df.shape)

## Kick controls and problem probes

In [ ]:
import methylcheck
from pathlib import Path
#filepath = Path('/Users/patriciagirardi/tutorial/GPL21145')
#df = methylcheck.load(filepath, format='beta_csv')

In [ ]:
# this code will print the criteria reason (either a publication or a type of issue, like Polymorphism)
# as well as the number of probes excluded for that reason 

criteria = ['Chen2013', 'Price2013', 'Naeem2014', 'DacaRoszak2015','Polymorphism',
             'CrossHybridization', 'BaseColorChange', 'RepeatSequenceElements']

print('450k probe exclusion criteria and number of probes excluded:')
for crit in criteria:
    print(crit, '--', len(methylcheck.list_problem_probes('450k', [crit])))

In [ ]:
# leave criteria undefined to list all problem probes for that array type
sketchy_probes_list = methylcheck.list_problem_probes(criteria=criteria, array='450k')

In [ ]:
df_filtered = df.loc[~ df.index.isin(sketchy_probes_list)]
df_filtered

In [ ]:
df_filtered = methylcheck.exclude_sex_control_probes(df_filtered, '450k', no_sex=True, no_control=True, verbose=True)
df_filtered

In [ ]:
methylcheck.mean_beta_compare(df, df_filtered, verbose=True)

## Nan handling

In [ ]:
df = df_filtered

In [ ]:
df.isna().any(), df.columns[df.isna().any()].tolist()

In [ ]:
#from sklearn.impute import KNNImputer
#imputer = KNNImputer(n_neighbors=2, weights="uniform")
#matrix = imputer.fit_transform(df.values)
#df = pd.DataFrame(matrix, columns = df.columns, index = df.index)

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
df = df.T #transpose as mean for col is calc
imputer = SimpleImputer(missing_values=np.NaN, strategy='mean') #mean for col
matrix = imputer.fit_transform(df.values)
df = pd.DataFrame(matrix, columns = df.columns, index = df.index)
df = df.T
df

## Kick low diff

In [ ]:
# kick low median deviation
from scipy.stats import median_abs_deviation

df = df.T
#for each row get median deviation
devs = []
for column in list(df.columns):
    col = df[column]
    devs.append(median_abs_deviation(col.values))
df.loc[len(df)] = devs

#sort and take top 10000 from 350000
devs.sort(reverse = True)
threshold_devs = devs[10000]
pd.Series(devs).hist()

In [ ]:
#filter for threshold
import numpy as np
mask = df.iloc[-1] > threshold_devs
keep = np.where(mask)[0]
df = df.iloc[:-1,keep.tolist()] #subset & kick last row

df

In [ ]:
df.reset_index(inplace = True)
df.rename(columns = {'index':'Sample_ID'}, inplace = True)
df

In [ ]:
#df.to_csv('Z:\HiWi\Popp\TCGA_Breast_2022\TCGA_BRCA_Methylation_450.csv')
df.to_csv('Z:\HiWi\Popp\TCGA_NSCLC_2022\LUNG\TCGA_LUNG_Methylation_450.csv')
#df.to_csv("Z:\HiWi\Popp\TCGA_NSCLC_2022\LUAD\DNA_methylation\TCGA_LUAD_Methylation_450.csv")

## Alternative: Mapping to gene names and chr info

In [ ]:
url = r'Z:\HiWi\Popp\TCGA_NSCLC_2022\LUAD\DNA_methylation\illuminaMethyl450_hg38_GDC'
df_mapping = pd.read_csv(url, sep='\t', index_col=0)
df_mapping

In [ ]:
df_merge = pd.merge(df, df_mapping, how='left', left_on='Composite Element REF', right_on='#id')
mask = (df_merge.gene.str.len() > 2) # kick dots and nans df_merge['gene'].isnull().any()
df_merge = df_merge.loc[mask]

In [ ]:
#extract one gene name from many
df_merge.gene = df_merge.gene.map(lambda x: x.split(',')[-1])

In [ ]:
df_merge.set_index('gene', inplace=True)
df_merge = df_merge.iloc[:,:-4]
df_merge

In [ ]:
#merge duplicate genes from extraction process
df_merge.sort_index(inplace=True)
df_merge = df_merge.groupby(df_merge.index).mean()

In [ ]:
df_merge = df_merge.T
df_merge = df_merge.reset_index()
df_merge.rename(columns = {'index':'Sample_ID'}, inplace = True)
print(df_merge.shape)
df_merge.dropna(how = 'all', axis=1, inplace=True) #dopped col if any is missing
df_merge.dropna(how = 'all', axis=0, inplace=True) #dopped col if any is missing
print(df_merge.shape)
df_merge

In [ ]:
df_merge.to_csv('Z:\HiWi\Popp\TCGA_NSCLC_2022\LUAD\DNA_methylation\TCGA_LUAD_Methylation_450.csv')

In [ ]:
# (9903 matchen df_prot_gene_mapping von 21169 gene names) --> need check!